# Data mining project
##Section 4
##Team members:
### Jana Ahmed Mohamed 202301800
### Ahmed Mohsen Abdelsattar 202301138

# **Imports**

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
import re

nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')

stop_words=stopwords.words('english')

stemmer = PorterStemmer()
wnl = WordNetLemmatizer()

!pip install python-terrier

import pyterrier as pt

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


# **Data exploration and cleaning (first dataset)**

In [ ]:
df= pd.read_csv('/content/amazon-products.csv')
df.head()

,timestamp,title,seller_name,brand,description,initial_price,final_price,currency,availability,reviews_count,...,root_bs_category,bs_category,bs_rank,badge,subcategory_rank,amazon_choice,images,product_details,prices_breakdown,country_of_origin
0,2023-08-08 00:00:00.000,Saucony Men's Kinvara 13 Running Shoe,Orv███tor███,Saucony,"When it comes to lightweight speed, nothing cr...",NaN,"""57.79""",USD,In Stock,702,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2023-08-09 00:00:00.000,Kishigo Premium Black Series Heavy Duty Unisex...,Ama███.co███,Kishigo,The Kishigo Premium Black Series Heavy Duty Ve...,NaN,"""28.5""",USD,In Stock,916,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2024-02-04 00:00:00.000,TWINSLUXES Solar Post Cap Lights Outdoor - Wat...,Twi███uxe███,TWINSLUXES,Solar Post Cap Lights Waterproof LED Fence Pos...,"""49.99""","""33.99""",USD,In Stock,3178,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2024-06-09 00:00:00.000,Accutire MS-4021B Digital Tire Pressure Gauge ...,Cit███ran███Dir██████,Accutire,About this item Heavy duty construction and ru...,1.795000000000000e+01,1.795000000000000e+01,USD,In Stock,8034,...,Automotive,Tire Repair Tools,50.0,NaN,"[{""subcategory_name"":""Automotive"",""subcategory...",False,NaN,NaN,NaN,NaN
4,2024-01-16 00:00:00.000,SAURA LIFE SCIENCE Adivasi Ayurvedic Neelgiri ...,PRA███ EN███PRI███,SAURA LIFE SCIENCE,This extraordinary fusion is designed to nouri...,"""1299""","""799""",INR,In stock,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df.columns

Index(['timestamp', 'title', 'seller_name', 'brand', 'description',
       'initial_price', 'final_price', 'currency', 'availability',
       'reviews_count', 'categories', 'asin', 'buybox_seller',
       'number_of_sellers', 'root_bs_rank', 'answered_questions', 'domain',
       'images_count', 'url', 'video_count', 'image_url', 'item_weight',
       'rating', 'product_dimensions', 'seller_id', 'date_first_available',
       'discount', 'model_number', 'manufacturer', 'department',
       'plus_content', 'upc', 'video', 'top_review', 'variations', 'delivery',
       'features', 'format', 'buybox_prices', 'parent_asin', 'input_asin',
       'ingredients', 'origin_url', 'bought_past_month', 'is_available',
       'root_bs_category', 'bs_category', 'bs_rank', 'badge',
       'subcategory_rank', 'amazon_choice', 'images', 'product_details',
       'prices_breakdown', 'country_of_origin'],
      dtype='object')

In [ ]:
df=df[["title", "brand", "description", "final_price", "currency", "availability", "reviews_count", "categories", "buybox_seller", "url", "image_url", "rating", "discount", "top_review"]]

In [ ]:
df.head()

,title,brand,description,final_price,currency,availability,reviews_count,categories,buybox_seller,url,image_url,rating,discount,top_review
0,Saucony Men's Kinvara 13 Running Shoe,Saucony,"When it comes to lightweight speed, nothing cr...","""57.79""",USD,In Stock,702,"[""Clothing, Shoes & Jewelry"",""Men"",""Shoes"",""At...",Orva Stores,https://www.amazon.com/Saucony-Mens-Kinvara-Ru...,https://m.media-amazon.com/images/I/71QeGmahUn...,4.6,NaN,I got the Saucony Men's Kinvara 13 Running Sho...
1,Kishigo Premium Black Series Heavy Duty Unisex...,Kishigo,The Kishigo Premium Black Series Heavy Duty Ve...,"""28.5""",USD,In Stock,916,"[""Tools & Home Improvement"",""Safety & Security...",Amazon.com,https://www.amazon.com/ML-Kishigo-Ultra-Cool-P...,https://m.media-amazon.com/images/I/41PUenVYhM...,4.6,NaN,Nice vest and comfort and great colors
2,TWINSLUXES Solar Post Cap Lights Outdoor - Wat...,TWINSLUXES,Solar Post Cap Lights Waterproof LED Fence Pos...,"""33.99""",USD,In Stock,3178,"[""Tools & Home Improvement"",""Lighting & Ceilin...",Twinsluxes,https://www.amazon.com/Solar-Post-Cap-Lights-O...,https://m.media-amazon.com/images/I/71kCgl9yzz...,4.2,-32%,These lovely solar lights not only light your ...
3,Accutire MS-4021B Digital Tire Pressure Gauge ...,Accutire,About this item Heavy duty construction and ru...,1.795000000000000e+01,USD,In Stock,8034,"[""Automotive"",""Tools & Equipment"",""Tire & Whee...",City Brands Direct,https://www.amazon.com/dp/B00080QHMM?th=1&psc=...,https://m.media-amazon.com/images/I/41UbPFLOVk...,4.4,NaN,My original Accutire MS-4021B worked well for ...
4,SAURA LIFE SCIENCE Adivasi Ayurvedic Neelgiri ...,SAURA LIFE SCIENCE,This extraordinary fusion is designed to nouri...,"""799""",INR,In stock,5,"[""Beauty"",""Hair Care"",""Hair Oils""]",PRANTH ENTERPRISE,https://www.amazon.in/SAURA-LIFE-SCIENCE-Ayurv...,https://m.media-amazon.com/images/I/31hYnlXSpe...,5.0,-38%,So best product for hair health.


In [ ]:
df.isna().sum()

,0
title,0
brand,1
description,2
final_price,3
currency,0
availability,6
reviews_count,0
categories,0
buybox_seller,74
url,0


In [ ]:
df.shape

(1000, 14)

In [ ]:
df["discount"].fillna(0, inplace=True)
df["top_review"].fillna("No Review", inplace=True)
df["buybox_seller"].fillna("Unknown", inplace=True)
df["image_url"].fillna("No Image found", inplace=True)
df["availability"].fillna("Unknown", inplace=True)
df.dropna(subset= ["final_price"], inplace=True)
df.dropna(subset= ["brand"], inplace=True)
df["description"].fillna("No Description", inplace=True)

<ipython-input-280-8e1c56ce04b5>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["discount"].fillna(0, inplace=True)
<ipython-input-280-8e1c56ce04b5>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.me

In [ ]:
df.duplicated().sum()

np.int64(0)

In [ ]:
df.describe()

,reviews_count,rating
count,996.000000,996.000000
mean,3254.828313,4.401908
std,11911.173338,0.312304
min,1.000000,2.500000
25%,129.750000,4.300000
50%,461.500000,4.400000
75%,1929.250000,4.600000
max,243907.000000,5.000000


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 996 entries, 0 to 999
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   title          996 non-null    object 
 1   brand          996 non-null    object 
 2   description    996 non-null    object 
 3   final_price    996 non-null    object 
 4   currency       996 non-null    object 
 5   availability   996 non-null    object 
 6   reviews_count  996 non-null    int64  
 7   categories     996 non-null    object 
 8   buybox_seller  996 non-null    object 
 9   url            996 non-null    object 
 10  image_url      996 non-null    object 
 11  rating         996 non-null    float64
 12  discount       996 non-null    object 
 13  top_review     996 non-null    object 
dtypes: float64(1), int64(1), object(12)
memory usage: 116.7+ KB


In [ ]:
df.isna().sum()

,0
title,0
brand,0
description,0
final_price,0
currency,0
availability,0
reviews_count,0
categories,0
buybox_seller,0
url,0


In [ ]:
df.shape

(996, 14)

In [ ]:
def floatDataType(text):
  text = re.sub(r'[\"]', '', text)
  text = re.sub(r'[\,]', '', text)

  return float(text)

df["final_price"]=df["final_price"].apply(floatDataType)

In [ ]:
df.head()

,title,brand,description,final_price,currency,availability,reviews_count,categories,buybox_seller,url,image_url,rating,discount,top_review
0,Saucony Men's Kinvara 13 Running Shoe,Saucony,"When it comes to lightweight speed, nothing cr...",57.79,USD,In Stock,702,"[""Clothing, Shoes & Jewelry"",""Men"",""Shoes"",""At...",Orva Stores,https://www.amazon.com/Saucony-Mens-Kinvara-Ru...,https://m.media-amazon.com/images/I/71QeGmahUn...,4.6,0,I got the Saucony Men's Kinvara 13 Running Sho...
1,Kishigo Premium Black Series Heavy Duty Unisex...,Kishigo,The Kishigo Premium Black Series Heavy Duty Ve...,28.50,USD,In Stock,916,"[""Tools & Home Improvement"",""Safety & Security...",Amazon.com,https://www.amazon.com/ML-Kishigo-Ultra-Cool-P...,https://m.media-amazon.com/images/I/41PUenVYhM...,4.6,0,Nice vest and comfort and great colors
2,TWINSLUXES Solar Post Cap Lights Outdoor - Wat...,TWINSLUXES,Solar Post Cap Lights Waterproof LED Fence Pos...,33.99,USD,In Stock,3178,"[""Tools & Home Improvement"",""Lighting & Ceilin...",Twinsluxes,https://www.amazon.com/Solar-Post-Cap-Lights-O...,https://m.media-amazon.com/images/I/71kCgl9yzz...,4.2,-32%,These lovely solar lights not only light your ...
3,Accutire MS-4021B Digital Tire Pressure Gauge ...,Accutire,About this item Heavy duty construction and ru...,17.95,USD,In Stock,8034,"[""Automotive"",""Tools & Equipment"",""Tire & Whee...",City Brands Direct,https://www.amazon.com/dp/B00080QHMM?th=1&psc=...,https://m.media-amazon.com/images/I/41UbPFLOVk...,4.4,0,My original Accutire MS-4021B worked well for ...
4,SAURA LIFE SCIENCE Adivasi Ayurvedic Neelgiri ...,SAURA LIFE SCIENCE,This extraordinary fusion is designed to nouri...,799.00,INR,In stock,5,"[""Beauty"",""Hair Care"",""Hair Oils""]",PRANTH ENTERPRISE,https://www.amazon.in/SAURA-LIFE-SCIENCE-Ayurv...,https://m.media-amazon.com/images/I/31hYnlXSpe...,5.0,-38%,So best product for hair health.


In [ ]:
df["final_price"].dtype

dtype('float64')

In [ ]:
df = df.reset_index(drop=True)

print(df.index)

RangeIndex(start=0, stop=996, step=1)


In [ ]:
df.head(25)

,title,brand,description,final_price,currency,availability,reviews_count,categories,buybox_seller,url,image_url,rating,discount,top_review
0,Saucony Men's Kinvara 13 Running Shoe,Saucony,"When it comes to lightweight speed, nothing cr...",57.79,USD,In Stock,702,"[""Clothing, Shoes & Jewelry"",""Men"",""Shoes"",""At...",Orva Stores,https://www.amazon.com/Saucony-Mens-Kinvara-Ru...,https://m.media-amazon.com/images/I/71QeGmahUn...,4.6,0,I got the Saucony Men's Kinvara 13 Running Sho...
1,Kishigo Premium Black Series Heavy Duty Unisex...,Kishigo,The Kishigo Premium Black Series Heavy Duty Ve...,28.50,USD,In Stock,916,"[""Tools & Home Improvement"",""Safety & Security...",Amazon.com,https://www.amazon.com/ML-Kishigo-Ultra-Cool-P...,https://m.media-amazon.com/images/I/41PUenVYhM...,4.6,0,Nice vest and comfort and great colors
2,TWINSLUXES Solar Post Cap Lights Outdoor - Wat...,TWINSLUXES,Solar Post Cap Lights Waterproof LED Fence Pos...,33.99,USD,In Stock,3178,"[""Tools & Home Improvement"",""Lighting & Ceilin...",Twinsluxes,https://www.amazon.com/Solar-Post-Cap-Lights-O...,https://m.media-amazon.com/images/I/71kCgl9yzz...,4.2,-32%,These lovely solar lights not only light your ...
3,Accutire MS-4021B Digital Tire Pressure Gauge ...,Accutire,About this item Heavy duty construction and ru...,17.95,USD,In Stock,8034,"[""Automotive"",""Tools & Equipment"",""Tire & Whee...",City Brands Direct,https://www.amazon.com/dp/B00080QHMM?th=1&psc=...,https://m.media-amazon.com/images/I/41UbPFLOVk...,4.4,0,My original Accutire MS-4021B worked well for ...
4,SAURA LIFE SCIENCE Adivasi Ayurvedic Neelgiri ...,SAURA LIFE SCIENCE,This extraordinary fusion is designed to nouri...,799.00,INR,In stock,5,"[""Beauty"",""Hair Care"",""Hair Oils""]",PRANTH ENTERPRISE,https://www.amazon.in/SAURA-LIFE-SCIENCE-Ayurv...,https://m.media-amazon.com/images/I/31hYnlXSpe...,5.0,-38%,So best product for hair health.
5,KASOTT Replacement Airpod Pro Ear Tip Premium ...,KASOTT,About this item [ Exclusive Customization ] Ma...,17.88,USD,In Stock,67,"[""Electronics"",""Headphones, Earbuds & Accessor...",KASOTT office,https://www.amazon.com/KASOTT-Replacement-Redu...,https://m.media-amazon.com/images/I/51PJLJF3tI...,4.3,-13%,My problem was the tips that come with AirPods...
6,Bio-Oil Skincare Body Oil (Natural) Serum for ...,Bio-Oil,Give your skin a break while stuck inside! It'...,24.95,USD,In Stock,6095,"[""Beauty & Personal Care"",""Skin Care"",""Face"",""...",Amazon.com,https://www.amazon.com/Bio-Oil-Skincare-Stretc...,https://m.media-amazon.com/images/I/41PBEKES5N...,4.6,$14.29$14.29 (36%),Using this for scars post surgery and have alr...
7,"crysting 13 Inch Sewing Box Three Layers, Plas...",crysting,About this item 🌟 High Quality Sewing Box 🌟 Ma...,20.99,USD,In Stock,210,"[""Arts, Crafts & Sewing"",""Organization, Storag...",GDDE-US,https://www.amazon.com/Plastic-Organizers-Mult...,https://m.media-amazon.com/images/I/61MDm2rPNp...,4.6,-9%,I was looking for a box to store my hair produ...
8,"Ridgid 62990 T-201 5"" Straight Auger",RIDGID,"The product is 62990 AUGER, T201 STRAIGHT 5/8 ...",25.38,USD,Only 5 left in stock - order soon,28,"[""Tools & Home Improvement"",""Power & Hand Tool...",Empire Rigging & Supply,https://www.amazon.com/Ridgid-62990-T-201-5-In...,https://m.media-amazon.com/images/I/61vEdS0dnn...,4.4,-7%,I was hoping this was a little thinner in diam...
9,Core 10 Women's All Day Comfort High-Waist Sid...,Core 10,Empowering women to reach their full potential...,23.92,USD,Only 1 left in stock - order soon,1538,"[""Clothing, Shoes & Jewelry"",""Women"",""Clothing...",Amazon.com,https://www.amazon.com/Core-10-Comfort-High-Wa...,https://m.media-amazon.com/images/I/718wu1uleP...,4.4,16,"Well made, fabric works wonders!"


# **Data exploration and cleaning (second dataset)**

In [ ]:
df3 = pd.read_csv("walmart-products.csv")
df3.head()

,timestamp,url,final_price,sku,currency,gtin,specifications,image_urls,top_reviews,rating_stars,...,sizes,colors,seller,other_attributes,customer_reviews,ingredients,initial_price,discount,ingredients_full,categories
0,2024-08-24 00:00:00.000,https://www.walmart.com/ip/Laura-Mercier-Cavia...,22.90,173530386,USD,7.361501e+11,"[{""name"":""Brand"",""value"":""Laura Mercier""},{""na...","[""https://i5.walmartimages.com/seo/Laura-Merci...","{""negative"":{},""positive"":{}}","{""five_stars"":2,""four_stars"":3,""two_stars"":1}",...,[],"[""Sugar Frost"",""Tuxedo""]",Wal███t.c███,"[{""name"":""Instructions"",""value"":""Apply directl...","[{""name"":""Jac███"",""rating"":5,""review"":""My only...","Cyclopentasiloxane, trimethylsiloxysilicate, s...",NaN,NaN,"[{""type"":""Ingredients"",""values"":""Cyclopentasil...","[""Beauty"",""Makeup"",""Eye Makeup"",""Eye Shadow"",""..."
1,2024-08-24 00:00:00.000,https://www.walmart.com/ip/Exultantex-Grey-Bla...,47.88,430528189,USD,7.710779e+11,"[{""name"":""Brand"",""value"":""Exultantex""},{""name""...","[""https://i5.walmartimages.com/seo/Exultantex-...","{""negative"":{""rating"":1,""review"":""Color not ac...","{""five_stars"":47,""four_stars"":4,""one_star"":4,""...",...,"[""50\"" x 54\"""",""50\"" x 63\"""",""50\"" x 84\"""",""50...","[""Black"",""Blue"",""Green"",""Gray"",""Natural(Ivory)...",Exu███nte███ome███,"[{""name"":""Fabric Care Instructions"",""value"":""M...","[{""name"":""Dana"",""rating"":5,""review"":""I love th...",NaN,70.80,$22.92,NaN,"[""Home"",""Decor"",""Curtains & Window Treatments""..."
2,2024-08-24 00:00:00.000,https://www.walmart.com/ip/Jessica-London-Wome...,33.24,6013308220,USD,4.651920e+11,"[{""name"":""Features"",""value"":""Easy Care""},{""nam...","[""https://i5.walmartimages.com/seo/Jessica-Lon...","{""negative"":{},""positive"":{}}","{""five_stars"":2}",...,"[""S"",""M"",""L"",""1X"",""2X"",""3X""]","[""Aqua Sea"",""Dark Olive Green"",""Dark Sapphire""...",Ful███aut███ran███,"[{""name"":""Fabric Care Instructions"",""value"":""M...","[{""name"":""Bev███y F███"",""rating"":5,""review"":""V...",NaN,37.99,$4.75,NaN,"[""Clothing"",""Womens Plus"",""Plus Size Tops"",""Pl..."
3,2024-08-24 00:00:00.000,https://www.walmart.com/ip/100-Cotton-King-Per...,49.99,161657830,USD,8.407081e+11,"[{""name"":""Brand"",""value"":""Simply Put""},{""name""...","[""https://i5.walmartimages.com/asr/d88fe658-b9...","{""negative"":{""rating"":1,""review"":""[This review...","{""five_stars"":78,""four_stars"":17,""one_star"":2,...",...,"[""Queen"",""King""]","[""Beige"",""Blue"",""Gray"",""Spa Blue""]",IC ███bal███c,"[{""name"":""Fabric Care Instructions"",""value"":""M...","[{""name"":""Jes███oge███"",""rating"":5,""review"":""[...",NaN,71.99,$22.00,NaN,"[""Home"",""Bedding"",""Duvet Covers"",""King Duvet C..."
4,2024-08-25 00:00:00.000,https://www.walmart.com/ip/Disney-Boys-Graphic...,12.99,5397071399,USD,4.600086e+11,"[{""name"":""Country of Origin - Textiles"",""value...","[""https://i5.walmartimages.com/seo/Disney-Boys...","{""negative"":{""rating"":1,""review"":""This size sh...","{""five_stars"":12,""one_star"":1,""three_stars"":1,...",...,"[""2T"",""3T"",""3T-4T"",""3-4 Years"",""4-5 Years"",""4-...","[""1#Yellow Micky"",""2#Pink Minnie"",""3#Blue Dona...",Pat███,NaN,"[{""name"":""Mary"",""rating"":5,""review"":""Love thes...",NaN,29.00,$16.01,NaN,"[""Clothing"",""Kids Clothing"",""Boys Clothing"",""B..."


In [ ]:
df3.columns

Index(['timestamp', 'url', 'final_price', 'sku', 'currency', 'gtin',
       'specifications', 'image_urls', 'top_reviews', 'rating_stars',
       'related_pages', 'available_for_delivery', 'available_for_pickup',
       'brand', 'breadcrumbs', 'category_ids', 'review_count', 'description',
       'product_id', 'product_name', 'review_tags', 'category_url',
       'category_name', 'category_path', 'root_category_url',
       'root_category_name', 'upc', 'tags', 'main_image', 'rating',
       'unit_price', 'unit', 'aisle', 'free_returns', 'sizes', 'colors',
       'seller', 'other_attributes', 'customer_reviews', 'ingredients',
       'initial_price', 'discount', 'ingredients_full', 'categories'],
      dtype='object')

In [ ]:
df3=df3[["product_name", "brand", "description", "final_price", "currency", "categories", "colors", "url", "image_urls", "rating", "discount", "top_reviews"]]

In [ ]:
df3.shape

(1000, 12)

In [ ]:
df3.isna().sum()

,0
product_name,0
brand,0
description,1
final_price,0
currency,0
categories,0
colors,0
url,0
image_urls,0
rating,0


In [ ]:
df3["discount"].fillna(0, inplace=True)
df3["description"].fillna("No Description", inplace=True)

<ipython-input-296-5d70a6edd7dc>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df3["discount"].fillna(0, inplace=True)
<ipython-input-296-5d70a6edd7dc>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3["discount"].fillna(0, inplace=True)
<ipython-input-296-5d70a6edd7dc>:2: FutureWarning: A value is trying to be set on a copy of a DataFra

In [ ]:
df3.duplicated().sum()

np.int64(0)

In [ ]:
df3["final_price"]

,final_price
0,22.90
1,47.88
2,33.24
3,49.99
4,12.99
...,...
995,2.08
996,13.49
997,16.99
998,32.39


# **Data exploration and cleaning (third dataset)**

In [ ]:
df4 = pd.read_csv("myntra_products_catalog.csv")

In [ ]:
df4.shape

(12491, 8)

In [ ]:
df4.head()

,ProductID,ProductName,ProductBrand,Gender,Price (INR),NumImages,Description,PrimaryColor
0,10017413,DKNY Unisex Black & Grey Printed Medium Trolle...,DKNY,Unisex,11745,7,"Black and grey printed medium trolley bag, sec...",Black
1,10016283,EthnoVogue Women Beige & Grey Made to Measure ...,EthnoVogue,Women,5810,7,Beige & Grey made to measure kurta with churid...,Beige
2,10009781,SPYKAR Women Pink Alexa Super Skinny Fit High-...,SPYKAR,Women,899,7,Pink coloured wash 5-pocket high-rise cropped ...,Pink
3,10015921,Raymond Men Blue Self-Design Single-Breasted B...,Raymond,Men,5599,5,Blue self-design bandhgala suitBlue self-desig...,Blue
4,10017833,Parx Men Brown & Off-White Slim Fit Printed Ca...,Parx,Men,759,5,"Brown and off-white printed casual shirt, has ...",White


In [ ]:
df4 = df4[["ProductName", "ProductBrand", "Gender", "Price (INR)", "Description", "PrimaryColor"]]

In [ ]:
df4["Price"]  = df4["Price (INR)"]*0.012

<ipython-input-303-10afe545b6ca>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df4["Price"]  = df4["Price (INR)"]*0.012


In [ ]:
df4.drop(["Price (INR)"], axis=1, inplace=True)

<ipython-input-304-5a2f6e4c4d39>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df4.drop(["Price (INR)"], axis=1, inplace=True)


In [ ]:
df4["Price"]

,Price
0,140.940
1,69.720
2,10.788
3,67.188
4,9.108
...,...
12486,15.588
12487,23.880
12488,7.224
12489,107.400


In [ ]:
df4.isna().sum()

,0
ProductName,0
ProductBrand,0
Gender,0
Description,0
PrimaryColor,894
Price,0


In [ ]:
df4["PrimaryColor"].fillna("Unknown", inplace=True)

<ipython-input-307-4ddfa672ad68>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df4["PrimaryColor"].fillna("Unknown", inplace=True)
<ipython-input-307-4ddfa672ad68>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df4["PrimaryColor"].fillna("Unknown", inplace=True)


In [ ]:
df4.isna().sum()

,0
ProductName,0
ProductBrand,0
Gender,0
Description,0
PrimaryColor,0
Price,0


In [ ]:
df4.duplicated().sum()

np.int64(311)

In [ ]:
df4.drop_duplicates(inplace=True)

In [ ]:
df4.duplicated().sum()

np.int64(0)

# **Data preprocessing**

In [ ]:
def preprocessing(text):
    text = text.lower()
    text = re.sub("[^a-z \:]+", "", text)
    result = []
    for word in text.split():
        if word not in stop_words:
            word = wnl.lemmatize(word, "n")
            word = wnl.lemmatize(word, "v")
            word = wnl.lemmatize(word, "a")
            word = stemmer.stem(word)
            result.append(word)
    return " ".join(result)

In [ ]:
df.head()

,title,brand,description,final_price,currency,availability,reviews_count,categories,buybox_seller,url,image_url,rating,discount,top_review
0,Saucony Men's Kinvara 13 Running Shoe,Saucony,"When it comes to lightweight speed, nothing cr...",57.79,USD,In Stock,702,"[""Clothing, Shoes & Jewelry"",""Men"",""Shoes"",""At...",Orva Stores,https://www.amazon.com/Saucony-Mens-Kinvara-Ru...,https://m.media-amazon.com/images/I/71QeGmahUn...,4.6,0,I got the Saucony Men's Kinvara 13 Running Sho...
1,Kishigo Premium Black Series Heavy Duty Unisex...,Kishigo,The Kishigo Premium Black Series Heavy Duty Ve...,28.50,USD,In Stock,916,"[""Tools & Home Improvement"",""Safety & Security...",Amazon.com,https://www.amazon.com/ML-Kishigo-Ultra-Cool-P...,https://m.media-amazon.com/images/I/41PUenVYhM...,4.6,0,Nice vest and comfort and great colors
2,TWINSLUXES Solar Post Cap Lights Outdoor - Wat...,TWINSLUXES,Solar Post Cap Lights Waterproof LED Fence Pos...,33.99,USD,In Stock,3178,"[""Tools & Home Improvement"",""Lighting & Ceilin...",Twinsluxes,https://www.amazon.com/Solar-Post-Cap-Lights-O...,https://m.media-amazon.com/images/I/71kCgl9yzz...,4.2,-32%,These lovely solar lights not only light your ...
3,Accutire MS-4021B Digital Tire Pressure Gauge ...,Accutire,About this item Heavy duty construction and ru...,17.95,USD,In Stock,8034,"[""Automotive"",""Tools & Equipment"",""Tire & Whee...",City Brands Direct,https://www.amazon.com/dp/B00080QHMM?th=1&psc=...,https://m.media-amazon.com/images/I/41UbPFLOVk...,4.4,0,My original Accutire MS-4021B worked well for ...
4,SAURA LIFE SCIENCE Adivasi Ayurvedic Neelgiri ...,SAURA LIFE SCIENCE,This extraordinary fusion is designed to nouri...,799.00,INR,In stock,5,"[""Beauty"",""Hair Care"",""Hair Oils""]",PRANTH ENTERPRISE,https://www.amazon.in/SAURA-LIFE-SCIENCE-Ayurv...,https://m.media-amazon.com/images/I/31hYnlXSpe...,5.0,-38%,So best product for hair health.


# **Corpus construction**

In [ ]:
with open("our_corpus.txt", "w") as f:

        df[df.columns] = df[df.columns].astype(str)
        df3[df3.columns] = df3[df3.columns].astype(str)
        df4[df4.columns] = df4[df4.columns].astype(str)

        for index, row in df.iterrows():
            f.write(f"Document {index + 1}:\n")
            for col in df.columns:
                column_name = col
                column_value = row[col]
                f.write(f"\t\t{column_name}: {column_value},\t\t\n")
            f.write("\n---\n\n")

        for index, row in df3.iterrows():
            f.write(f"Document {index + 1}:\n")
            for col in df3.columns:
                column_name = col
                column_value = row[col]
                f.write(f"\t\t{column_name}: {column_value},\t\t\n")
            f.write("\n---\n\n")

        for index, row in df4.iterrows():
            f.write(f"Document {index + 1}:\n")
            for col in df4.columns:
                column_name = col
                column_value = row[col]
                f.write(f"\t\t{column_name}: {column_value},\t\t\n")
            f.write("\n---\n\n")

print("Corpus created successfully!")


Corpus created successfully!


In [ ]:
import pandas as pd

corpus_file = "our_corpus.txt"

documents_preprocess = []
documents_without_preprocess = []
doc_id = 0
with open(corpus_file, "r") as f:
    doc_text = ""

    for line in f:
        line = line.strip()
        if line.startswith("Document"):
            if doc_text:
                documents_without_preprocess.append({"docno": str(doc_id), "text": doc_text})
                doc_text = preprocessing(doc_text)
                documents_preprocess.append({"docno": str(doc_id), "text": doc_text})
                doc_id += 1
            doc_text = ""
        elif line and not line.startswith("---"):
            doc_text += " " + line

    if doc_text:
      documents_without_preprocess.append({"docno": str(doc_id), "text": doc_text})
      doc_text = preprocessing(doc_text)
      documents_preprocess.append({"docno": str(doc_id), "text": doc_text})

df = pd.DataFrame(documents_preprocess)
df2 = pd.DataFrame(documents_without_preprocess)


print(df.head())
print(df2.head())




  docno                                               text
0     0  title: sauconi men kinvara run shoe brand: sau...
1     1  title: kishigo premium black seri heavi duti u...
2     2  title: twinslux solar post cap light outdoor w...
3     3  title: accutir msb digit tire pressur gaug val...
4     4  title: saura life scienc adivasi ayurved neelg...
  docno                                               text
0     0   title: Saucony Men's Kinvara 13 Running Shoe,...
1     1   title: Kishigo Premium Black Series Heavy Dut...
2     2   title: TWINSLUXES Solar Post Cap Lights Outdo...
3     3   title: Accutire MS-4021B Digital Tire Pressur...
4     4   title: SAURA LIFE SCIENCE Adivasi Ayurvedic N...


In [ ]:
df2.shape

(14176, 2)

# **Inverted Indexing**

In [ ]:
if not pt.java.started():
    pt.java.init()

In [ ]:
df["docno"] = list(range(df.shape[0]))
df["docno"] = df["docno"].astype(str)

indexer = pt.DFIndexer("./myIndex", overwrite=True)
index_ref = indexer.index(df["text"], df["docno"])
index = pt.IndexFactory.of(index_ref)

print(index.getCollectionStatistics().toString())

<ipython-input-318-d903cefb6a13>:4: DeprecationWarning: Call to deprecated class DFIndexer. (use pt.terrier.IterDictIndexer().index(dataframe.to_dict(orient='records')) instead) -- Deprecated since version 0.11.0.
  indexer = pt.DFIndexer("./myIndex", overwrite=True)


Number of documents: 14176
Number of terms: 34134
Number of postings: 583345
Number of fields: 0
Number of tokens: 852468
Field names: []
Positions:   false



In [ ]:
lexicon = index.getLexicon()
for word in lexicon:
    print(word.getKey())

Streaming output truncated to the last 5000 lines.
suppliestap
suppliestissu
suppliesvacuum
suppliesw
supplieswallpap
supplieswast
supplieswat
supplieswrit
suppliment
supplimenti
supplimentrec
suppo
support
supportampcoverag
supportcontrast
supportelast
supporteva
supportfit
supporti
supportleath
supportlightweight
supportnbsp
supportnrgi
supportour
supportposit
supportpow
supportpuma
supportquot
supportrain
supportrubb
supportsleep
supportslip
supporttextur
supportth
supportthi
supporttpu
supposedli
suppress
supprt
suprem
suprema
supremequ
sur
sure
suregrip
sureplea
sureremov
surew
surf
surfac
surfacedont
surfacefacilit
surfacehpasusacerdel
surfaceil
surfacemeasur
surfaceproduct
surfacesfg
surfacesfix
surfaceslow
surfacessmooth
surfacesthei
surfacesthi
surfaceth
surfact
surfboard
surfer
surfvibr
surg
surgeri
surgic
surinam
surpass
surplic
surpri
surprisi
surprisingli
surround
surroundingsfabr
surveillancesurveil
surviv
survivalhand
survivei
suscept
sushiyi
suspect
suspen
suspend
suspe

In [ ]:
for x in index.getLexicon():
  print("%s : %s " % (x.getKey(), x.getValue().toString()))

Streaming output truncated to the last 5000 lines.
suppliestap : term17828 Nt=2 TF=2 maxTF=1 @{0 528389 5} 
suppliestissu : term3359 Nt=2 TF=2 maxTF=1 @{0 528393 7} 
suppliesvacuum : term12246 Nt=1 TF=1 maxTF=1 @{0 528397 3} 
suppliesw : term17174 Nt=1 TF=1 maxTF=1 @{0 528399 5} 
supplieswallpap : term2176 Nt=1 TF=1 maxTF=1 @{0 528402 1} 
supplieswast : term5485 Nt=2 TF=2 maxTF=1 @{0 528403 3} 
supplieswat : term11483 Nt=1 TF=1 maxTF=1 @{0 528407 7} 
supplieswrit : term4041 Nt=6 TF=6 maxTF=1 @{0 528410 1} 
suppliment : term6868 Nt=2 TF=4 maxTF=3 @{0 528420 7} 
supplimenti : term6839 Nt=1 TF=1 maxTF=1 @{0 528425 3} 
supplimentrec : term6851 Nt=1 TF=1 maxTF=1 @{0 528427 3} 
suppo : term2062 Nt=37 TF=43 maxTF=3 @{0 528429 3} 
support : term122 Nt=372 TF=653 maxTF=16 @{0 528482 1} 
supportampcoverag : term21756 Nt=1 TF=1 maxTF=1 @{0 528861 2} 
supportcontrast : term33083 Nt=2 TF=2 maxTF=1 @{0 528864 0} 
supportelast : term32322 Nt=1 TF=1 maxTF=1 @{0 528870 0} 
supporteva : term33952 Nt=1 T

In [ ]:
def search_term(term_original):

    term = preprocessing(term_original)
    print(f"\nSearching for: {term_original}")
    print(f"Term after preprocessing is: {term}")


    try:
        pointer = index.getLexicon()[term]
        postings_id=[]
        for posting in index.getInvertedIndex().getPostings(pointer):
           postings_id.append(str(posting.getId()))
        print(f"Term {term} was found in {len(postings_id)} documents with document IDs {postings_id}\nThese documents are:")
        for posting in index.getInvertedIndex().getPostings(pointer):
          id = posting.getId()
          details = df2[df2['docno'] == str(id)]["text"].values[0]
          print(f"\nDocument ID: {id}, Length: {posting.getDocumentLength()}\nDetails: {details}")

    except KeyError:
        print(f" Term '{term}' not found in the index.")

In [ ]:
df.head()

,docno,text
0,0,title: sauconi men kinvara run shoe brand: sau...
1,1,title: kishigo premium black seri heavi duti u...
2,2,title: twinslux solar post cap light outdoor w...
3,3,title: accutir msb digit tire pressur gaug val...
4,4,title: saura life scienc adivasi ayurved neelg...


In [ ]:
search_term('charger')


Searching for: charger
Term after preprocessing is: charger
Term charger was found in 32 documents with document IDs ['14', '73', '125', '128', '346', '410', '437', '453', '487', '493', '545', '574', '582', '590', '609', '626', '635', '642', '655', '728', '736', '768', '808', '838', '849', '871', '889', '910', '911', '965', '7879', '11249']
These documents are:

Document ID: 14, Length: 134
Details:  title: 2Packs 18V Battery and Charger Combo for 18-Volt Cordless Tools Battery and P118B Charger, Cell9102 18V Battery Capacity Output 3.0Ah, brand: Cell9102, description: About this item The Cell9102 Advantage : Professional power tool battery manufacturing factory, with dozens of research and development team. So that you can use the battery more efficiently to bring convenience to your life What You Get : Cell9102 2Packs replacement 18V battery and Charger Starter Kit, 18V 3000mAh lithium Ion P108 and P118B 18V battery charger Ultra-High Capacity : This item replacement 18V battery has

In [ ]:
def final_search_inverted():
  search = input("Enter your search term: ")
  search_term(search)

# **TF-IDF**

In [ ]:
tfidf_retr = pt.BatchRetrieve(index, controls={"wmodel": "TF_IDF"}, num_results=30)

<ipython-input-325-d86883b83c68>:1: DeprecationWarning: Call to deprecated class BatchRetrieve. (use pt.terrier.Retriever() instead) -- Deprecated since version 0.11.0.
  tfidf_retr = pt.BatchRetrieve(index, controls={"wmodel": "TF_IDF"}, num_results=30)


In [ ]:
query = preprocessing("hotel")

tfidf_results = tfidf_retr.search(query)
tfidf_results = tfidf_results.merge(df2, left_on='docno', right_on='docno')
tfidf_results.rename(columns={'text': 'description'}, inplace=True)

tfidf_results

,qid,docid,docno,rank,score,query,description
0,1,1788,1788,0,4.773939,hotel,product_name: 400 Thread Count White Cotton S...
1,1,745,745,1,3.309530,hotel,title: RUVANTI Full Size Sheets - Silky Soft ...
2,1,303,303,2,3.200981,hotel,"title: Ambesonne Fishing Fitted Sheet, Fishin..."
3,1,1057,1057,3,3.169462,hotel,product_name: Feather & Stitch 2 Piece Pillow...
4,1,269,269,4,2.962614,hotel,title: Ambesonne Western Fitted Sheet & Pillo...
5,1,1725,1725,5,2.856266,hotel,product_name: Full Size Luxury Extrasoft 100%...
6,1,852,852,6,2.843792,hotel,"title: 100% Egyptian Cotton Pillowcases, 600 ..."
7,1,1530,1530,7,2.843507,hotel,product_name: Olympic Queen Size Luxury Extra...
8,1,846,846,8,2.781384,hotel,title: Presto 03510 Ceramic FlipSide Belgian ...
9,1,1817,1817,9,2.728012,hotel,product_name: American Soft Linen Luxury 6 Pi...


In [ ]:
print(type(tfidf_results))

<class 'pandas.core.frame.DataFrame'>


In [ ]:
def final_search_tfidf():
  search_t = input("Enter your search term: ")
  query = preprocessing(search_t)
  tfidf_results = tfidf_retr.search(query)
  tfidf_results = tfidf_results.merge(df2, left_on='docno', right_on='docno')
  tfidf_results.rename(columns={'text': 'description'}, inplace=True)
  return tfidf_results


# **BM25**

In [ ]:
bm25_retr = pt.BatchRetrieve(index, controls={"wmodel": "BM25"}, num_results=30)

<ipython-input-329-6bd3fc45911d>:1: DeprecationWarning: Call to deprecated class BatchRetrieve. (use pt.terrier.Retriever() instead) -- Deprecated since version 0.11.0.
  bm25_retr = pt.BatchRetrieve(index, controls={"wmodel": "BM25"}, num_results=30)


In [ ]:
query = preprocessing("hotel")

bm25_results = bm25_retr.search(query)
bm25_results = bm25_results.merge(df2, left_on='docno', right_on='docno')
bm25_results.rename(columns={'text': 'description'}, inplace=True)

bm25_results

,qid,docid,docno,rank,score,query,description
0,1,1788,1788,0,8.725611,hotel,product_name: 400 Thread Count White Cotton S...
1,1,745,745,1,6.049024,hotel,title: RUVANTI Full Size Sheets - Silky Soft ...
2,1,303,303,2,5.850623,hotel,"title: Ambesonne Fishing Fitted Sheet, Fishin..."
3,1,1057,1057,3,5.793014,hotel,product_name: Feather & Stitch 2 Piece Pillow...
4,1,269,269,4,5.414946,hotel,title: Ambesonne Western Fitted Sheet & Pillo...
5,1,1725,1725,5,5.220567,hotel,product_name: Full Size Luxury Extrasoft 100%...
6,1,852,852,6,5.197767,hotel,"title: 100% Egyptian Cotton Pillowcases, 600 ..."
7,1,1530,1530,7,5.197246,hotel,product_name: Olympic Queen Size Luxury Extra...
8,1,846,846,8,5.083700,hotel,title: Presto 03510 Ceramic FlipSide Belgian ...
9,1,1817,1817,9,4.986148,hotel,product_name: American Soft Linen Luxury 6 Pi...


In [ ]:
def final_search_BM25():
  search_t = input("Enter your search term: ")
  query = preprocessing(search_t)
  bm25_results = bm25_retr.search(query)
  bm25_results = bm25_results.merge(df2, left_on='docno', right_on='docno')
  bm25_results.rename(columns={'text': 'description'}, inplace=True)
  return bm25_results


# **Testing**

In [ ]:
final_search_inverted()

Enter your search term: car

Searching for: car
Term after preprocessing is: car
Term car was found in 74 documents with document IDs ['24', '39', '61', '75', '108', '109', '151', '189', '192', '202', '223', '232', '314', '346', '366', '387', '388', '405', '421', '424', '432', '435', '445', '478', '495', '504', '533', '534', '543', '547', '565', '567', '584', '586', '589', '610', '632', '637', '650', '655', '686', '709', '713', '726', '759', '768', '783', '798', '800', '837', '863', '865', '871', '897', '903', '919', '943', '947', '962', '965', '966', '1099', '1119', '1124', '1464', '1489', '1536', '1659', '1895', '9619', '10849', '10917', '11148', '11573']
These documents are:

Document ID: 24, Length: 232
Details:  title: Car Underglow Lights,4PCs,180 LED RGB 8 Million Colors Music Mode,Car Under Glow Neon Accent Strip Lights,Waterproof Smart LED Exterior Lights Kit for Car, SUV, Truck,with APP Control,DC 12V, brand: H HESHECEIN, description: HESHECEIN Low-power Design Neon Exterior 

In [ ]:
output = final_search_tfidf()
output

Enter your search term: horse


,qid,docid,docno,rank,score,query,description
0,1,354,354,0,11.146454,hors,title: Thrush Treatment for Horses - Pure Sol...
1,1,1924,1924,1,3.250492,hors,product_name: NYX Jumbo Eye Pencil (Color : #...
2,1,833,833,2,3.206397,hors,title: The Step Adjustable High Step Aerobic ...
3,1,871,871,3,1.664028,hors,"title: Jump Starter with Air Compressor, YABE..."


In [ ]:
output = final_search_BM25()
output

Enter your search term: car


,qid,docid,docno,rank,score,query,description
0,1,232,232,0,13.187246,car,"title: 12V Car Heater,200W Portable Windshiel..."
1,1,1659,1659,1,12.685213,car,product_name: Fast Go 50 Mph Mini RC Car Off ...
2,1,495,495,2,12.498645,car,title: BDK Croc Skin Faux Leather Car Seat Co...
3,1,189,189,3,12.081017,car,title: ASLONG 5PCS Angel Wings Front and Back...
4,1,965,965,4,11.659826,car,title: Digital GPS Speedometer Universal Car ...
5,1,962,962,5,11.443974,car,title: BDK carXS Fresh Two-Tone Carpet Floor ...
6,1,919,919,6,11.328545,car,"title: GENORTH Dog Car Seats for Small Dogs,U..."
7,1,759,759,7,11.309399,car,title: 3 Piece Set Dog Seat Belt Retractable ...
8,1,637,637,8,11.195194,car,title: for Toyota Highlander Windshield Snow ...
9,1,24,24,9,10.814235,car,"title: Car Underglow Lights,4PCs,180 LED RGB ..."
